[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adan-rs/amd/blob/main/notebooks/09_Prueba_chi_cuadrada.ipynb)

# Prueba Chi cuadrada

Para ilustrar este tema utilizaremos el archivo "enigh2020". Importa la biblioteca de *Pandas* y carga esta base de datos en un DataFrame con el nombre *df*

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#df = pd.read_excel('../data/enigh2020.xlsx')
df = pd.read_excel('https://github.com/adan-rs/amd/raw/main/data/enigh2020.xlsx')

In [ ]:
# Crea una tabla cruzada para relacionar "sexo_jefe" con "est_socio"
tabla = pd.crosstab(df['sexo_jefe'], df['est_socio'], margins=False)
tabla

## Prueba Chi cuadrado

*¿Para qué se utiliza?*
La prueba chi cuadrada de independencia se emplea para determinar si existe una asociación estadísticamente significativa entre dos variables cualitativas (categóricas). Es especialmente útil en estudios de contingencia, como encuestas o experimentos, donde se desea evaluar si la distribución de una variable depende del valor de otra.
Estos son algunos casos ilustrativos de uso:
- Una empresa desea saber si existe una relación entre el perfil de riesgo del inversionista (conservador, moderado, agresivo) y el tipo de instrumento financiero que elige (bonos, acciones, fondos de inversión).
- La dirección de recursos humanos quiere saber si la satisfacción laboral (alta, media, baja) está asociada al departamento en el que labora el empleado (ventas, producción, administración, logística).
- Una empresa minorista quiere conocer si existe relación entre el grupo de edad del consumidor (18–30, 31–45, 46+) y el tipo de promoción preferida (descuento directo, 2x1, puntos de lealtad).

*Variables consideradas:*
La prueba requiere dos variables cualitativas, ya sean nominales (sin orden natural, como el género o el tipo de producto) u ordinales (con cierto orden, como nivel educativo o grado de satisfacción). Cada variable debe tener un número limitado de categorías, lo que permite organizar los datos en una tabla de contingencia.

*Requisitos o supuestos:*
Para que los resultados de la prueba sean válidos, deben cumplirse ciertas condiciones relacionadas con las frecuencias esperadas en la tabla de contingencia:
- En una tabla de 2 x 2, ninguna celda debe tener una frecuencia esperada menor a 5.
- En tablas de mayor dimensión (como 3 x 3 o más), se permite que hasta el 20% de las celdas tenga una frecuencia esperada menor a 5, pero ninguna debe tener una frecuencia esperada menor a 1.
- Los datos deben provenir de una muestra aleatoria y las observaciones deben ser independientes entre sí.

*Hipótesis planteadas:*
- Hipótesis nula (H0): Las variables son independientes, es decir, no existe asociación entre ellas.
- Hipótesis alternativa (H1): Las variables son dependientes, es decir, existe una relación entre ellas.


In [ ]:
import numpy as np
from scipy.stats import chi2_contingency

Un ejemplo de la aplicación a una tabla de contingencia es:  
`chi2, p, dof, expected = chi2_contingency(tabla)`  
Esta instrucción arrojará  los siguientes resultados:
- El estadístico de prueba
- El p-valor
- Los grados de libertad
- La tabla de frecuencias esperadas


Para imprimir estos resultados:  
```
print('Estadístico de prueba:', chi2) 
print('Valor p:', p)  
print('Grados de libertad:', dof)  
print('Tabla de frecuencias esperadas')   
print(expected)
```


In [ ]:
# Realiza una prueba chi cuadrada a la tabla obtenida
chi2, p, dof, expected = chi2_contingency(tabla)

# Mostrar resultados
print('Estadístico de prueba: ', chi2) 
print('Valor p: ', p)  
print('Grados de libertad: ', dof)  
print('Tabla de frecuencias esperadas:')   
print(expected)

In [ ]:
alfa = 0.05
if p <=alfa:
    print('Las variables son dependientes (Se rechaza la hipótesis nula)')
else:
    print('Las variables son independientes (No se rechaza la hipótesis nula)')

Podemos crear una función que integre los pasos anteriores

In [ ]:
from scipy.stats import chi2_contingency

def prueba_chi_cuadrada(df, var1, var2, alfa=0.05, imprimir=True):
    """
    Realiza una prueba chi cuadrada de independencia entre dos variables categóricas.
    Retorna un diccionario con resultados de la prueba.
    """
    # Verificar existencia de variables
    if var1 not in df.columns or var2 not in df.columns:
        raise ValueError(f"Una o ambas variables no existen en el DataFrame: {var1}, {var2}")

    # Crear tabla de contingencia
    tabla_contingencia = pd.crosstab(df[var1], df[var2])

    # Realizar prueba chi cuadrada
    chi2, p_valor, grados_libertad, frecuencias_esperadas = chi2_contingency(tabla_contingencia)

    # Calcular proporción de celdas con frecuencia esperada menor a 5
    celdas_menor_5 = np.sum(frecuencias_esperadas < 5)
    total_celdas = frecuencias_esperadas.size
    prop_freq_menor_5 = celdas_menor_5 / total_celdas

    # Preparar advertencia sobre frecuencias esperadas si es necesario
    advertencia = None
    if prop_freq_menor_5 > 0.2:
        advertencia = ("ADVERTENCIA: Más del 20% de las celdas tienen frecuencias esperadas menores a 5. "
                       "Los resultados de la prueba podrían no ser confiables.")  
    
    # Determinar resultado
    resultado = ('Las variables son dependientes (Se rechaza la hipótesis nula)' 
        if p_valor < alfa 
        else 'Las variables son independientes (No se rechaza la hipótesis nula)')

    # Imprimir resultados
    print(f'\nTabla de contingencia:\n{tabla_contingencia}\n')
    print(f'Estadístico de prueba: {chi2:.4f}')
    print(f'Valor p: {p_valor:.4f}')
    print(f'Grados de libertad: {grados_libertad}')
    print('\nTabla de frecuencias esperadas:')
    print(pd.DataFrame(
        frecuencias_esperadas,
        index=tabla_contingencia.index,
        columns=tabla_contingencia.columns).round(4))
    print(f'\nProporción de celdas con frecuencia esperada menor a 5: {prop_freq_menor_5:.2%}')
    print(f'({celdas_menor_5} de {total_celdas} celdas)')
    if advertencia:
        print(f'\n{advertencia}')
    print(f'\nNivel de significancia: {alfa}')
    print(f'Resultado: {resultado}')

    # Retornar p-valor
    return p_valor

In [ ]:
resultados = prueba_chi_cuadrada(df, 'sexo_jefe', 'est_socio')

Ejemplo de reporte de metodología y resultados:
>“Se realizó una prueba chi cuadrada para evaluar la relación entre el sexo del jefe de familia y el estrato socioeconómico. Se encontró una relación significativa entre ambas variables, χ²(3) = 12.3959, p = 0.006.”

## Alternativa
La prueba exacta de Fisher es una alternativa apropiada en tablas cuando el número de observaciones es pequeño o hay celdas con frecuencias esperadas menores a 5. Se puede aplicar utilizando el módulo de *fisher_exact* en la biblioteca *scipy.stats*.

## Ejercicio

La base de datos "aerolineas" contiene información de publicaciones en X (antes Twitter) que han sido clasificados por su sentimiento (positivo, neutral y negativo) y que etiquetan a diferentes aerolíneas. Tu tarea es investigar si existe una relación significativa entre el sentimiento de los mensajes y las aerolíneas mencionadas. 

Este análisis te permitirá determinar si la distribución de los sentimientos (positivo, neutral, negativo) difiere entre las diferentes aerolíneas, sugiriendo una posible asociación entre el tipo de sentimiento expresado en los mensajes y la aerolínea mencionada. Los resultados de esta prueba estadística podrían proporcionar información valiosa sobre la percepción del servicio de las distintas aerolíneas entre los usuarios de esta red social.


In [ ]:
df2 = pd.read_excel('https://github.com/adan-rs/amd/raw/main/data/aerolineas.xlsx')